In [1]:
# Notebook header

import os
import numpy as np
from pybat.core import Cathode
from pymatgen import Composition, Element, Specie

# Workflow submission notebook - Honeycomb structures

From this notebook we'll submit the configuration workflows for the honeycomb-restricted configurations of the Sn-doped structures of Li$_{1.2}$Mn$_{0.8}$O$_2$.

> **Note**: As the code below relies on my own pre-alpha code ([pybat](https://github.com/mbercx/pybat)), it is possible that the code has changed since this jupyter notebook was set up. Please contact me via [researchgate](https://www.researchgate.net/profile/Marnik_Bercx) if you run into any issues, or have questions.

## Generating the configurations

The cells below are used to generate the honeycomb-like structures. This is done in 5 steps:

1. Start from the pristine primitive structure of O3-Li2MnO3, with space group C2/m.
2.  Replace the Li in the Li/Mn layer by a placeholder element, e.g. Lr. This is just to keep track of which sites correspond to the Li sites in the honeycomb layer.
3. Make a 2×2×2 supercell. This size is chosen to get as close as possible to the experimental composition of Li, without having to consider a unit cell size that is prohibitively large.

In [2]:
# Set up the base cathode from with to generate the random honeycomb structures, based on the primitive Li2MnO3 cell.

honey_cat = Cathode.from_file("o3-abc-afm_Li2MnO3_prim.json")
honey_cat.remove_site_property("velocities")
honey_cat.replace(0, "Lr", properties={"magmom": 3.014}) # Placeholder element
honey_cat.replace(5, "Mn", properties={"magmom": 3.014}) # Stick to ferromagnetic ordering
honey_cat.sort()
honey_cat.make_supercell([2, 2, 2])

4. Use the Cathode.get cation configurations() method to generate honeycomb-like struc- tures by substituting the Lr by Li and Mn, restricting the Li concentration of the final configurations to closely match that of the experimental samples.


In [3]:
# Generate all configurations possible by substituting the placeholder element (Lr) by Li and Mn

Li_configurations = honey_cat.get_cation_configurations(
    substitution_sites=[i for i, s in enumerate(honey_cat) if s.specie == Element("Lr")],
    cation_list=["Li", "Mn"],
    sizes=[1,],
    concentration_restrictions={"Li": (0.30, 0.305)}, # This corresponds to about Li1.2Mn0.8O2 stoichiometry
    max_configurations=1e2
)

5. Finally, for each of the Li/Mn configurations, generate all Li/Mn/Sn configurations with a specified Sn concentration. The cell below corresponds to x = 0.042:

In [4]:
# Generate all possible Sn-doped structures for a specific choice of Sn-concentration

total_conf_list = []

# For each Li-Mn configuration, generate all Sn-Mn configurations
for conf in Li_configurations:
    total_conf_list.extend(conf.get_cation_configurations(
    substitution_sites=[i for i, s in enumerate(conf) if s.specie == Element("Mn")],
    cation_list=["Mn", "Sn"],
    sizes=[1,],
    concentration_restrictions={"Sn": (0.010, 0.012)}, # Sn-concentration is determined here
    max_configurations=None
))

In [5]:
len(total_conf_list)

47

We can see that for the lowest possible Sn concentration in the 2x2x2 supercell, we find 47 possible Li/Mn/Sn configurations.

---

## Submitting the workflow

The cells below generate the configuration workflow based on the configuration list constructed above. Note that the `launchpad` and `base_dir` depend on the cluster you are running the calculations on.

In [6]:
from pybat.workflow.workflows import get_wf_configurations
from pybat.config import load_config

lpad = load_config("launchpad", "leibniz")
base_dir = "/scratch/antwerpen/202/vsc20248/batteries"

In [7]:
wf = get_wf_configurations(
    structure=honey_cat,
    directory=os.path.join(base_dir, str(total_conf_list[0].composition).replace(" ", "")),
    substitution_sites=[],
    sizes=[],
    configuration_list=total_conf_list,
    functional=("scanVdW", {}),
    in_custodian=True,
    number_nodes=4
)

In [8]:
lpad.add_wf(wf)

2020-03-03 18:51:33,605 INFO Added a workflow. id_map: {-1: 11}


{-1: 11}